In [4]:
import torch
from micrograd.engine import Value
from graphviz import Digraph

Function for drawing nodes

In [5]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ data %.4f | grad %.4f }" % (n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

Testing of Engine for Values

In [25]:
# Using Value 

# Forward Pass
x = Value(-4.0)
z = 2 * x + 2 + x
q = z.relu() + z * x
h = (z * z).relu()
y = h + q + q * x

print(f"""" 
      x = {x},
      z = {z},
      q = {q},
      h = {h},
      y = {y}
      """)

#Backward pass
y.backward()
xmg, ymg = x, y

print(f"""" 
      xmg = {xmg},
      ymg = {ymg}
      """)

" 
      x = Value(data=-4.0, grad=0),
      z = Value(data=-10.0, grad=0),
      q = Value(data=40.0, grad=0),
      h = Value(data=100.0, grad=0),
      y = Value(data=-20.0, grad=0)
      
" 
      xmg = Value(data=-4.0, grad=46.0),
      ymg = Value(data=-20.0, grad=1)
      


In [26]:
# Using PyTorch
# Forward Pass
x = torch.Tensor([-4.0]).double()
x.requires_grad = True
z = 2 * x + 2 + x
q = z.relu() + z * x
h = (z * z).relu()
y = h + q + q * x

print(f"""" 
      x = {x},
      z = {z},
      q = {q},
      h = {h},
      y = {y}
      """)

y.backward()
xpt, ypt = x, y

print(f"""" 
      xpt = {xpt}
      ypt = {ypt}
      """)


" 
      x = tensor([-4.], dtype=torch.float64, requires_grad=True),
      z = tensor([-10.], dtype=torch.float64, grad_fn=<AddBackward0>),
      q = tensor([40.], dtype=torch.float64, grad_fn=<AddBackward0>),
      h = tensor([100.], dtype=torch.float64, grad_fn=<ReluBackward0>),
      y = tensor([-20.], dtype=torch.float64, grad_fn=<AddBackward0>)
      
" 
      xpt = tensor([-4.], dtype=torch.float64, requires_grad=True)
      ypt = tensor([-20.], dtype=torch.float64, grad_fn=<AddBackward0>)
      


In [27]:
a = Value(-4.0)
b = Value(2.0)
c = a + b
d = a * b + b**3
c += c + 1
c += 1 + c + (-a)
d += d * 2 + (b + a).relu()
d += 3 * d + (b - a).relu()
e = c - d
f = e**2
g = f / 2.0
g += 10.0 / f
g.backward()
amg, bmg, gmg = a, b, g

print(f"""
    amg = {amg},
    bmg = {bmg},
    gmg = {gmg}  
""")


    amg = Value(data=-4.0, grad=138.83381924198252),
    bmg = Value(data=2.0, grad=645.5772594752186),
    gmg = Value(data=24.70408163265306, grad=1)  



In [28]:
a = torch.Tensor([-4.0]).double()
b = torch.Tensor([2.0]).double()
a.requires_grad = True
b.requires_grad = True
c = a + b
d = a * b + b**3
c = c + c + 1
c = c + 1 + c + (-a)
d = d + d * 2 + (b + a).relu()
d = d + 3 * d + (b - a).relu()
e = c - d
f = e**2
g = f / 2.0
g = g + 10.0 / f
g.backward()
apt, bpt, gpt = a, b, g

print(f"""
    apt = {apt},
    bpt = {bpt},
    gpt = {gpt}  
""")


    apt = tensor([-4.], dtype=torch.float64, requires_grad=True),
    bpt = tensor([2.], dtype=torch.float64, requires_grad=True),
    gpt = tensor([24.7041], dtype=torch.float64, grad_fn=<AddBackward0>)  

